# Comparison of rent and sale prices

In [1]:
%load_ext nb_black

from matplotlib import pyplot as plt

%matplotlib inline
plt.rcParams["figure.figsize"] = (15, 10)

import pandas as pd

pd.set_option("display.precision", 4)

<IPython.core.display.Javascript object>

## 1) Data cleaning

In [2]:
date = "2020-04-10"
rent_filepath = f"../data/mieten{date}.csv"
sale_filepath = f"../data/kaufen{date}.csv"
rent_df = pd.read_csv(rent_filepath, sep=";", low_memory=False)
sale_df = pd.read_csv(sale_filepath, sep=";", low_memory=False)
pd.set_option("display.max_columns", max(rent_df.shape[1], sale_df.shape[1]))

<IPython.core.display.Javascript object>

###### remove duplicates:

In [3]:
rent_df = rent_df.drop_duplicates(subset="obj_scoutId")
sale_df = sale_df.drop_duplicates(subset="obj_scoutId")

<IPython.core.display.Javascript object>

###### remove columns with missing values:

In [4]:
rent_df = rent_df[rent_df.columns[~rent_df.isnull().any()]]
sale_df = sale_df[sale_df.columns[~sale_df.isnull().any()]]

<IPython.core.display.Javascript object>

## 2) Feature engineering

###### add features:

In [35]:
rent_df["rent_per_m2"] = rent_df["obj_baseRent"] / rent_df["obj_livingSpace"]

rent_df["links"] = "https://www.immobilienscout24.de/expose/" + rent_df[
    "obj_scoutId"
].astype(str)

sale_df["salesprice_per_m2"] = sale_df["obj_purchasePrice"] / sale_df["obj_livingSpace"]

sale_df["links"] = "https://www.immobilienscout24.de/expose/" + sale_df[
    "obj_scoutId"
].astype(str)

<IPython.core.display.Javascript object>

###### select features:

In [5]:
rent_select_columns = [
    "links",
    "obj_baseRent",
    "obj_livingSpace",
    "obj_noRooms",
    "obj_zipCode",
    "obj_regio1",
    "obj_regio2",
    "obj_regio3",
]
rent_select_df = rent_df[rent_select_columns].copy()

sale_select_columns = [
    "links",
    "obj_purchasePrice",
    "obj_livingSpace",
    "obj_noRooms",
    "obj_zipCode",
    "obj_regio1",
    "obj_regio2",
    "obj_regio3",
]
sale_select_df = sale_df[sale_select_columns].copy()

<IPython.core.display.Javascript object>

###### group and combine rent and sale into new dataframe

In [25]:
group = "obj_regio3"
min_count = 3
aggregate = "median"
rent_feature = "rent_per_m2"
sale_feature = "salesprice_per_m2"

rent_group = rent_select_df.groupby(group)
rent_filt = rent_group[rent_feature].agg([aggregate, "count"])["count"] >= min_count
rent_group_df = rent_group[rent_feature].agg([aggregate, "count"])[rent_filt]

sale_group = sale_select_df.groupby(group)
sale_filt = sale_group[sale_feature].agg([aggregate, "count"])["count"] >= min_count
sale_group_df = sale_group[sale_feature].agg([aggregate, "count"])[sale_filt]

combined_df = pd.concat(
    [
        rent_group_df[aggregate],
        rent_group_df["count"],
        sale_group_df[aggregate],
        sale_group_df["count"],
    ],
    axis=1,
    keys=[rent_feature, "rent_count", sale_feature, "sale_count",],
).copy()

<IPython.core.display.Javascript object>

###### remove rows with missing values:

In [26]:
combined_df = combined_df.dropna()

<IPython.core.display.Javascript object>

###### yearly rent to salesprice ratios:

In [27]:
combined_feature = "yearly_ROI"
combined_df[combined_feature] = (
    combined_df[rent_feature] * 12 / (combined_df[sale_feature])
)
combined_feature = "salesprice_in_rents"
combined_df[combined_feature] = combined_df[sale_feature] / (
    combined_df[rent_feature] * 12
)

<IPython.core.display.Javascript object>

## 3) Results

In [28]:
combined_df = combined_df.sort_values("yearly_ROI", ascending=False)
combined_df.head(500)

,rent_per_m2,rent_count,salesprice_per_m2,sale_count,yearly_ROI,salesprice_in_rents
Gößnitz,4.5446,10.0,25.3984,8.0,2.1472,0.4657
Marxloh,6.0000,43.0,142.2239,17.0,0.5062,1.9753
Greiz,4.9606,50.0,155.3672,3.0,0.3831,2.6100
Dürrröhrsdorf_Dittersbach,5.1154,5.0,296.1347,3.0,0.2073,4.8242
Frohburg,5.4879,9.0,320.7834,3.0,0.2053,4.8710
...,...,...,...,...,...,...
Nordhorn,6.7950,8.0,1654.1540,4.0,0.0493,20.2863
Bad_Liebenzell,9.3684,5.0,2281.1739,4.0,0.0493,20.2913
Finthen,12.3159,12.0,3000.0000,9.0,0.0493,20.2989
Schinkel,8.0000,9.0,1948.9479,6.0,0.0493,20.3015


<IPython.core.display.Javascript object>

In [33]:
rent_filt_col = "obj_zipCode"
rent_filt_arg = 72074

rent_filt = rent_select_df[rent_filt_col] == rent_filt_arg
rent_select_df[rent_filt]

,obj_scoutId,obj_baseRent,obj_livingSpace,obj_noRooms,obj_zipCode,obj_regio1,obj_regio2,obj_regio3,rent_per_m2
22609,117067170,865.0,63.85,2.0,72074,Baden_Württemberg,Tübingen_Kreis,Tübingen,13.5474
27573,94758279,475.0,30.50,1.5,72074,Baden_Württemberg,Tübingen_Kreis,Tübingen,15.5738
34325,117139032,900.0,72.00,2.0,72074,Baden_Württemberg,Tübingen_Kreis,Tübingen,12.5000
40930,117015876,400.0,21.00,1.0,72074,Baden_Württemberg,Tübingen_Kreis,Tübingen,19.0476
55948,116358365,350.0,11.00,1.0,72074,Baden_Württemberg,Tübingen_Kreis,Tübingen,31.8182
58453,116058550,470.0,22.76,1.0,72074,Baden_Württemberg,Tübingen_Kreis,Tübingen,20.6503
81669,104538924,300.0,19.00,1.0,72074,Baden_Württemberg,Tübingen_Kreis,Tübingen,15.7895
85029,90113522,620.0,45.00,2.0,72074,Baden_Württemberg,Tübingen_Kreis,Tübingen,13.7778


<IPython.core.display.Javascript object>